# Fine Tuning Llama Model for Generation

**Authors:** Matías Arévalo, Pilar Guerrero, Moritz Goebbels, Tomás Lock, Allan Stalker  
**Date:** January – May 2025  

## Purpose
Fine-tunes a LLaMA based language model using LoRA adapters to generate spam messages based on the preprocessed dataset with generated prompts.

## CONSIDERATION
Due to some updates done to the `unsloth` package in early April, some outputs might differ from the ones used for our model. Because of this, the `output` used from this code in the following notebooks is going to be provided in the `Outputs` folder in the repository, in the subfolder called `fine_tuned_llama` as `checkpoint_1500` so it can be used for replication.  
In case one runs the notebook again, some differences will be found.

## Environment Setup

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
!pip install --upgrade unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-bhunmjfv/unsloth_da1a0e52a4d64739beb4e88641f92183
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-bhunmjfv/unsloth_da1a0e52a4d64739beb4e88641f92183
  Resolved https://github.com/unslothai/unsloth.git to commit ae0e5626106a7895f06a4516d298cb06f4710723
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Import Libraries

In [ ]:
import os
import torch
import pandas as pd
from trl import SFTTrainer
from datasets import Dataset
from unsloth import FastLanguageModel
from transformers import TextStreamer
from unsloth import FastLanguageModel
from transformers import TrainingArguments

<ipython-input-3-1fa78adb5a21>:6: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.6.0+cu124)
    Python  3.11.12 (you have 3.11.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


## Model Setup  
In this section, we configure the key parameters for loading the LLaMA-based mode we will be fine tuning later on the notebook.

We set the following parameters:  
- `max_seq_length` : used to define the maximum number of tokens the model will take as input. For efficient training we set it to `2048`, however it is important to note that LLaMA-3 models can handle up to 8k tokens.
- `dtype` : used to specify the data types for the models weights. To automatically select the appropriate type based on the hardware we set it to `none`.
- `load_in_4bit` : used to enable 4-bit optimization, which significantly reduces memory usage. As it is good when fine-tuning models on limited-resource environments, we set it to `True`.


Additionally, we included a list of a set of available 4-bit fine tuned models from the Unsloth library that can be used if different variants are needed throughout the notebook.

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]

## Load Pre-trained Model  
In this section we load the pre-trained LLaMA-3 8B model using the Unsloth library, applying the previously defined settings for sequence length, data type, and quantization.

For this we use the `FastLanguageModel.from_pretrained()` function to initialize bothe the model and tokenizer.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

## Fine Tuning the Model  
Throughout this section, the different steps to fine tune the loaded model will be made.

### Fine Tuning Preparation  
Here we load and preprocess the dataset used for fine-tuning.

The file to use is called `final_scam_prompt_dataset.csv`, which should be located in the `data/` folder.

**Step 1**: Load the dataset and preview

In [ ]:
df = pd.read_csv("../../data/final_scam_prompt_dataset.csv")

In [ ]:
df = pd.read_csv("final_scam_prompt_dataset.csv")
df.head()

,label,clean_message,prompt
0,spam,free entry in 2 a wkly comp to win fa cup fina...,Generate a prompt targeting individuals intere...
1,spam,freemsg hey there darling it s been 3 week s...,Generate a prompt that targets individuals see...
2,spam,winner!! as a valued network customer you have...,Generate a prompt targeting individuals who ha...
3,spam,had your mobile num months or more? u r enti...,Generate a prompt targeting individuals who ha...
4,spam,"six chances to win cash! from num to num ,...",Generate a prompt for a text message promotion...


**Step 2**: Drop the `label` column.

We drop this column because for two reasons:  
- Every entry in this dataset is `spam`  
- It is not going to be used in this notebook

In [ ]:
df = df.drop(columns=["label"])

**Step 3**: Rename the `message` column

In [ ]:
df.rename(columns={"clean_message": "completion"}, inplace=True)

**Step 4**: Merge `prompt` and `completions`

In [ ]:
EOS_TOKEN = tokenizer.eos_token or ""
def combine_prompt(row):
    return row["prompt"] + "\n\n" + row["completion"] + EOS_TOKEN
df["full_text"] = df.apply(combine_prompt, axis=1)

**Step 5**: Tokenize input texts

In [ ]:
tokenised = tokenizer(
    df["full_text"].tolist(),
    padding="max_length",
    truncation=True,
    max_length=max_seq_length,
    return_tensors="np",
)

**Step 6**: Build final dataset

In [ ]:
final_data = {
    "input_ids": tokenised["input_ids"],
    "attention_mask": tokenised["attention_mask"],
    "labels": tokenised["input_ids"].copy()
}
dataset = Dataset.from_dict(final_data)

### Apply LoRA Adapters  
Here we apply lightweight LoRA adapters to the base model, allowing parameter efficient fine tuning.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.4.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Model Training Configuration  
Here we set the different characteristics and parameters needed for the model we are making to be trained on.

In [ ]:
training_args = TrainingArguments(
    output_dir="llama_finetuning_outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    max_steps=1500,
    learning_rate=2e-4,
    logging_steps=1,
    bf16=torch.cuda.is_bf16_supported(),
    fp16=not torch.cuda.is_bf16_supported(),
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
)

### Initialize the Trainer

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args,
)

<ipython-input-17-3372ecd41825>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML (num_proc=12):   0%|          | 0/8049 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=12):   0%|          | 0/8049 [00:00<?, ? examples/s]

### Train and Save the Model  
We are going to save the model as `llama_finetuning_outputs` within the `Outputs`folder, which is at the same level as the `data` folder.

In the case of this repository, we will include the last checkpoint of this training model in the `Outputs` folder within the `llama_finetuning_outputs` subfolder. This checkpoint, saved as `checkpoint-1500`, is the one we are going to use in further notebooks and can be directly loaded into them without needed to run the following training process.

In [ ]:
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'
trainer.train()
trainer.save_model("../../Outputs/llama_finetuning_outputs")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,049 | Num Epochs = 2 | Total steps = 1,500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,8.753200
2,7.736500
3,6.603100
4,6.558400
5,6.397700
6,6.374400
7,5.159100
8,5.151300
9,5.538800
10,4.176500


## Load Fine Tuned Model
In this section, we will set up the environment following the previous setup made and load the `checkpoint-1500` found within the `llama_finetuning_outputs` folder, which should be located within the `Outputs` folder to apply it.

### Load Base Model (Same as the one for Training)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.4.5: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

### Apply LoRA Adapters (Same as the one for Training)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.4.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Load Fine Tuned Weights
This is the output we saved during the training of the model, it should be:  
- Called `llama_finetuning_outputs`
- Located in `Outputs` folder

However, for this example we are going to be using the checkpoint we have manually loaded into the `Outputs` folder, the location and file name is:
- File name: `checkpoint-1500`
- Location: `Outputs/llama_finetuning_outputs`

In [ ]:
model.load_adapter("../../Outputs/llama_finetuning_outputs/checkpoint-1500", adapter_name="default")

<All keys matched successfully>

## Testing Fine Tuned Model: Generate Sample Outputs

Use the fine tuned model to generate spam messages based on custom prompts to see how it generates messages.

### Test 1: 150 Maximum New Tokens

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
prompt = "Generate a spam message targeting people interested in cryptocurrency investments."
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

_ = model.generate(
    **inputs,
    streamer=streamer,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
)

 Make sure to mention the potential for high returns and include some personal information.

i invest money into bitcoin i get  num  return as profit this is my second time earning from bitcoin investment i started with an initial deposit of  num  and got  num   num  in 7 days after trading click on the link below for more information     


### Test 2: 200 Maximum New Tokens

In [ ]:
prompt = "Generate a spam message targeting individuals interested in investment opportunities, including cryptocurrency, stocks, and forex, with an emphasis on high returns.\n\nMessage:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

_ = model.generate(
    **inputs,
    streamer=streamer,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
)

 return to the sender.  num  
